In [1]:
import xarray as xr
from sklearn_xarray import wrap
from dask_ml.preprocessing import StandardScaler
import intake
from dask.distributed import Client
# client = Client("tcp://127.0.0.1:30730",processes=False)
client = Client(processes=False)
client

/home8/homedir3/perso/gmaze/miniconda3/envs/obidam36/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: inproc://134.246.146.54/1724/1 Dashboard: http://localhost:48827/status,Cluster Workers: 1 Cores: 32 Memory: 67.48 GB


In [8]:
catalog_url = 'https://raw.githubusercontent.com/obidam/m2poc2019/master/m2poc_data_catalog.yml'
ds = intake.Catalog(catalog_url).argo_global_homogeneous_sdl.read_chunked()
feat_0 = ds.TEMP.sel(DEPTH=0, method='nearest')
feat_1 = ds.TEMP.sel(DEPTH=-1000, method='nearest')
RES = xr.concat([feat_0, feat_1], dim='feature').chunk(chunks={'N_PROF':10000})
RES = RES.rename('ARGO')
# Remove obsolete attributes:
for a in RES.attrs.copy(): 
    del RES.attrs[a]
# Add feature names:
RES.attrs['feature_0'] = "%s at %0.2fm depth"%(feat_0.name,feat_0['DEPTH'])
RES.attrs['feature_1'] = "%s at %0.2fm depth"%(feat_1.name,feat_1['DEPTH'])
# We're good:
print(RES)

<xarray.DataArray 'ARGO' (feature: 2, N_PROF: 128910)>
dask.array<shape=(2, 128910), dtype=float32, chunksize=(1, 10000)>
Coordinates:
    LATITUDE   (N_PROF) float32 dask.array<shape=(128910,), chunksize=(10000,)>
    LONGITUDE  (N_PROF) float32 dask.array<shape=(128910,), chunksize=(10000,)>
    TIME       (N_PROF) datetime64[ns] dask.array<shape=(128910,), chunksize=(10000,)>
    DEPTH      (feature) float32 dask.array<shape=(2,), chunksize=(2,)>
Dimensions without coordinates: feature, N_PROF
Attributes:
    feature_0:  TEMP at 0.00m depth
    feature_1:  TEMP at -1000.00m depth


In [14]:
X = xr.DataArray(RES.values.T, dims=['sample','feature'])
print(X.values.shape)

(128910, 2)


In [17]:
# scaler = wrap(StandardScaler(), sample_dim='N_PROF')
scaler = wrap(StandardScaler(), sample_dim='sample')
%time scaler.fit(X)
print(scaler.estimator_.mean_)
print(scaler.estimator_.scale_)
scaler.transform(X)

CPU times: user 3 ms, sys: 1e+03 µs, total: 4 ms
Wall time: 2.3 ms
[-9.5641155e-08  2.3862941e-07]
[1.        0.9999999]


<xarray.DataArray (sample: 128910, feature: 2)>
array([[-1.551811, -1.29166 ],
       [-1.82948 , -1.280055],
       [-1.588464, -1.29136 ],
       ...,
       [-1.656578, -0.882205],
       [-0.824594, -0.895902],
       [-1.067999, -0.828581]], dtype=float32)
Dimensions without coordinates: sample, feature

In [40]:
# Artificially increase the size of the dataset:
# X = xr.concat([RES] * 1000, dim='N_PROF')

distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('xarray-DEPTH-f8a0f43e839eead34589b4da32f05b82', 0)": []}
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('xarray-DEPTH-f8a0f43e839eead34589b4da32f05b82', 0)": []}
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('xarray-DEPTH-f8a0f43e839eead34589b4da32f05b82', 0)": []}
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('xarray-DEPTH-f8a0f43e839eead34589b4da32f05b82', 0)": []}
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('xarray-DEPTH-f8a0f43e839eead34589b4da32f05b82', 0)": []}
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('xarray-DEPTH-f8a0f43e839eead34589b4da32f05b82', 0)": []}
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('xarray-DEPTH-f8a0f43e839eead34589b4da32f05b82', 0)": []}
distributed.client - WARNING - Couldn't gather 1 keys, rescheduling {"('xarray-DEPTH-f8a0f43e839e

In [41]:
print(X)
print('This dataset holds: %.3f GB' % (X.nbytes / 1e9))

<xarray.DataArray 'ARGO' (N_FEATURES: 2, N_PROF: 128910000)>
dask.array<shape=(2, 128910000), dtype=float32, chunksize=(1, 10000)>
Coordinates:
    DEPTH      (N_FEATURES) float32 0.0 -1000.0
    LATITUDE   (N_PROF) float32 dask.array<shape=(128910000,), chunksize=(10000,)>
    LONGITUDE  (N_PROF) float32 dask.array<shape=(128910000,), chunksize=(10000,)>
    TIME       (N_PROF) datetime64[ns] dask.array<shape=(128910000,), chunksize=(10000,)>
Dimensions without coordinates: N_FEATURES, N_PROF
Attributes:
    feature_0:  TEMP at 0.00m depth
    feature_1:  TEMP at -1000.00m depth
This dataset holds: 1.031 GB


In [ ]:
scaler = wrap(StandardScaler(),sample_dim='N_PROF')
%time scaler.fit(X)
print(scaler.estimator_.mean_
print(scaler.estimator_.scale_)